![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 4 Notebook - AutoAI Notebook v1.21.1

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - snapml


In [18]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install autoai-libs==1.16.1 | tail -n 1
!pip install scikit-learn==1.1.1 | tail -n 1
!pip install 'lale>=0.7,<0.8' | tail -n 1
!pip install 'snapml==1.13.1' | tail -n 1

Filter warnings for this notebook.

In [19]:
import warnings

warnings.filterwarnings('ignore')

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [20]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='c0864091-aed4-41dc-b3b4-ef85dacd5609'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/7672ce4b-7078-4ed3-9c04-33e28fc58503/wml_data/78a49f69-0e1d-4280-9267-5e94b1fdc4d4/data/automl',
        model_location='auto_ml/7672ce4b-7078-4ed3-9c04-33e28fc58503/wml_data/78a49f69-0e1d-4280-9267-5e94b1fdc4d4/data/automl/model.zip',
        training_status='auto_ml/7672ce4b-7078-4ed3-9c04-33e28fc58503/wml_data/78a49f69-0e1d-4280-9267-5e94b1fdc4d4/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [21]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='MISSED_PAYMENT',
    holdout_size=0.1,
    scoring='accuracy',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='0d00bfee-97c4-4f15-a3ff-d9b75d0e1124',
    positive_label='Paid',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[]
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [22]:
api_key = '14qK2UldM1i25REfFm3HcH9y23_NVL-6taz3pgKjrnC9'

In [23]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [24]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. 

It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.


In [25]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [26]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TA2
from snapml import SnapBoostingMachineClassifier

#### Pre-processing & Estimator.

In [27]:
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        3, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
        29, 30, 31, 32, 33, 40, 41,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "char_str", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "char_str", "char_str",
        "char_str", "char_str", "char_str", "char_str", "int_num", "int_num",
        "int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 34, 35, 36, 37, 38, 39]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "int_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        3, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
        29, 30, 31, 32, 33, 40, 41, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 34, 35,
        36, 37, 38, 39,
    ],
)
ta1 = TA1(
    fun=np.square,
    name="square",
    datatypes=["numeric"],
    feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "CUSTOMER_ID", "OVERDUE_BALANCE", "BASE_USAGE", "CREDIT_HISTORY",
        "ALTERNATE_USAGE", "STANDING_CHARGE", "BASE_CHARGE",
        "ALTERNATE_CHARGE", "LEVY", "TAX", "TOTAL_NET", "TOTAL_TO_PAY", "AGE",
        "IS_REGISTERED_FOR_ALERTS", "OWNS_HOME", "COMPLAINTS",
        "HAS_THERMOSTAT", "HAS_HOME_AUTOMATION", "PV_ZONING", "WIND_ZONING",
        "SMART_METER_COMMENTS", "IS_CAR_OWNER", "HAS_EV", "HAS_PV",
        "HAS_WIND", "EBILL", "IN_WARRANTY", "CITY", "MARITAL_STATUS",
        "EDUCATION", "SEGMENT", "EMPLOYMENT", "BUILDING_TYPE",
        "BILLING_MONTH", "RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_BASE_USAGE_VS_AVG_LOOKBACK_WINDOW",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_AVG_LOOKBACK_WINDOW",
        "RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_AVG_LOOKBACK_WINDOW",
        "NUM_MISSED_PAYMENTS_LOOKBACK_WINDOW", "BILLING_MONTH_NUMBER",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 42),
    additional_col_count_to_keep=30,
    ptype="classification",
)
ta2 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "CUSTOMER_ID", "OVERDUE_BALANCE", "BASE_USAGE", "CREDIT_HISTORY",
        "ALTERNATE_USAGE", "STANDING_CHARGE", "BASE_CHARGE",
        "ALTERNATE_CHARGE", "LEVY", "TAX", "TOTAL_NET", "TOTAL_TO_PAY", "AGE",
        "IS_REGISTERED_FOR_ALERTS", "OWNS_HOME", "COMPLAINTS",
        "HAS_THERMOSTAT", "HAS_HOME_AUTOMATION", "PV_ZONING", "WIND_ZONING",
        "SMART_METER_COMMENTS", "IS_CAR_OWNER", "HAS_EV", "HAS_PV",
        "HAS_WIND", "EBILL", "IN_WARRANTY", "CITY", "MARITAL_STATUS",
        "EDUCATION", "SEGMENT", "EMPLOYMENT", "BUILDING_TYPE",
        "BILLING_MONTH", "RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_BASE_USAGE_VS_AVG_LOOKBACK_WINDOW",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_AVG_LOOKBACK_WINDOW",
        "RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_AVG_LOOKBACK_WINDOW",
        "NUM_MISSED_PAYMENTS_LOOKBACK_WINDOW", "BILLING_MONTH_NUMBER",
        "square(CUSTOMER_ID)", "square(OVERDUE_BALANCE)",
        "square(BASE_USAGE)", "square(ALTERNATE_USAGE)",
        "square(BASE_CHARGE)", "square(ALTERNATE_CHARGE)", "square(LEVY)",
        "square(TAX)", "square(TOTAL_NET)", "square(TOTAL_TO_PAY)",
        "square(AGE)", "square(RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH)",
        "square(RATIO_THIS_MONTH_BASE_USAGE_VS_AVG_LOOKBACK_WINDOW)",
        "square(RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH)",
        "square(RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_AVG_LOOKBACK_WINDOW)",
        "square(RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_LAST_MONTH)",
        "square(RATIO_THIS_MONTH_TOTAL_TO_PAY_VS_AVG_LOOKBACK_WINDOW)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 42),
    additional_col_count_to_keep=30,
    ptype="classification",
)
snap_boosting_machine_classifier = SnapBoostingMachineClassifier(
    class_weight="balanced",
    gpu_ids=[0],
    learning_rate=0.01,
    max_max_depth=3,
    min_max_depth=3,
    num_round=73,
    random_state=33,
)


#### Pipeline.

In [28]:
pipeline = make_pipeline(
    union,
    numpy_permute_array,
    ta1,
    fs1_0,
    ta2,
    fs1_1,
    snap_boosting_machine_classifier,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [29]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [30]:
pipeline.fit(train_X.values, train_y.values.ravel());

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [31]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

0.9142857142857143


In [32]:
pipeline.predict(test_X.values[:5])

array(['Paid', 'Missed Payment', 'Paid', 'Paid', 'Paid'], dtype='<U32')

<a id="saving"></a>
## Store the model

In this section you will learn how to store the trained model.

In [33]:
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'P4 - Pretrained AutoAI pipeline'
}

stored_model_details = wml_client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [34]:
stored_model_details

{'entity': {'hybrid_pipeline_software_specs': [{'id': 'b4246d87-d8b3-5621-9fea-76a8ea452616',
    'name': 'autoai-kb_rt23.1-py3.10'}],
  'label_column': 'MISSED_PAYMENT',
  'pipeline': {'id': '8955c3c8-abba-452e-a384-971e1fe64fea'},
  'schemas': {'input': [{'fields': [{'name': 'CUSTOMER_ID', 'type': 'integer'},
      {'name': 'OVERDUE_BALANCE', 'type': 'double'},
      {'name': 'BASE_USAGE', 'type': 'double'},
      {'name': 'CREDIT_HISTORY', 'type': 'other'},
      {'name': 'ALTERNATE_USAGE', 'type': 'double'},
      {'name': 'STANDING_CHARGE', 'type': 'double'},
      {'name': 'BASE_CHARGE', 'type': 'double'},
      {'name': 'ALTERNATE_CHARGE', 'type': 'double'},
      {'name': 'LEVY', 'type': 'double'},
      {'name': 'TAX', 'type': 'double'},
      {'name': 'TOTAL_NET', 'type': 'double'},
      {'name': 'TOTAL_TO_PAY', 'type': 'double'},
      {'name': 'AGE', 'type': 'integer'},
      {'name': 'IS_REGISTERED_FOR_ALERTS', 'type': 'integer'},
      {'name': 'OWNS_HOME', 'type': 'inte

<a id="deployment"></a>
## Create online deployment

You can use commands bellow to promote the model to space and create online deployment (web service).

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space for organizing the assets for deploying and scoring the model. If you do not have an existing space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space, following these steps:

- Click **New Deployment Space**.
- Create an empty space.
- Select Cloud Object Storage.
- Select Watson Machine Learning instance and press **Create**.
- Copy `space_id` and paste it below.

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

#### Prepare online deployment

#### Test online deployment

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `wml_client.deployments.delete(deployment_id)` command.
To list the existing web services, use `wml_client.deployments.list()`.

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___